# SCALING UP DATA ANALYSIS (Part I): FORMATS AND METHODS

## FRAMEWORKS TO TEST: Pandas, Polars, DuckDB and Vaex

In [ ]:
##  DATA:
##  https://krono.act.uji.es/IDIA/loans.csv.gz

In [ ]:
# !pip install polars -q #new version is 0.19

In [ ]:
# !pip install duckdb -q

In [1]:
%pip install vaex

  Using cached vaex-4.17.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached vaex-core-4.17.1.tar.gz (2.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached vaex_astro-0.9.3-py3-none-any.whl.metadata (345 bytes)
  Using cached vaex_hdf5-0.14.1-py3-none-any.whl.metadata (321 bytes)
  Using cached vaex_viz-0.5.4-py3-none-any.whl.metadata (345 bytes)
  Using cached vaex_server-0.9.0-py3-none-any.whl.metadata (430 bytes)
  Using cached vaex_jupyter-0.8.2-py3-none-any.whl.metadata (517 bytes)
  Using cached vaex_ml-0.18.3-py3-none-any.whl.metadata (485 bytes)
  Using cached astropy-6.1.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached aplus-0.11.0-py3-none-any.whl
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached progressbar2-4.5.0-py3-none-any.whl.metadata

In [6]:
import pandas as pd
import polars as pl
import duckdb
import vaex

ModuleNotFoundError: No module named 'vaex'

In [ ]:
import os

## LOADING & TRANSFORMING DATA

Profiling code (guide):
https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.07-Timing-and-Profiling.ipynb

In [ ]:
!wget https://krono.act.uji.es/IDIA/loans.csv.gz

--2024-10-14 16:48:40--  https://krono.act.uji.es/IDIA/loans.csv.gz
Resolving krono.act.uji.es (krono.act.uji.es)... 150.128.97.37
Connecting to krono.act.uji.es (krono.act.uji.es)|150.128.97.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82624417 (79M) [application/x-gzip]
Saving to: ‘loans.csv.gz’

loans.csv.gz        100%[===================>]  78.80M  7.06MB/s    in 14s     

2024-10-14 16:48:55 (5.53 MB/s) - ‘loans.csv.gz’ saved [82624417/82624417]



In [ ]:
!gunzip loans.csv.gz

### PANDAS

In [ ]:
df = pd.read_csv('./loans.csv')  #use timeit for several executions

In [ ]:
df.head()

,uuid,use_uuid,bicycle_uuid,anchor_uuid,date_created,type_access
0,30300405f1fb4a2e8ebeef4377b2fea1,NaN,fb135edf71a74ca48085d6fa4b719c76,f239242f253c43f69f0fd655cdb9e898,2017-04-12 17:06:28,UNKNOWN
1,3030296398be45ca9ec387d146c10ef2,024cbe02bf494e4f9125ede5a21db1a8,57ebb2215f2a45f0a1c0c5a97f6fde79,1042bbe5c80549c987e51f04d1cb9285,2017-05-11 18:03:53,UNKNOWN
2,303035331fe14b8797a8488c85b7d26a,5edc5b39c72047a6b34e17426495c99a,30ccded725e94a6a88dfd9456b4cd6d0,f8c081df1b524c1f8db348da809ceff3,2017-04-11 21:02:45,UNKNOWN
3,30303bfde83e450e93580ca739f1cad2,NaN,1d06622f183d485297aa91a0da25ff40,64b4da4811624d77a3bf15225ed9a50f,2017-05-09 08:24:05,UNKNOWN
4,30303e3ff4b84b4da053b7b605a0d9d4,NaN,66b8a22235a345b78d19899cd37bc277,f1d0b0627c7d4cd698c16e94a7a7ab2b,2017-04-22 08:31:42,UNKNOWN


**Using the Apache Parquet format**

https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_parquet.html

In [ ]:
if not os.path.exists('loans.parquet'):
  df.to_parquet('loans.parquet') #, compression='gzip')
  #transform to parquet format -> compression='snappy' by default


In [ ]:
!ls -GFlash *.*

235M -rw-r--r-- 1 root 235M Oct 13  2023 loans.csv
 74M -rw-r--r-- 1 root  74M Oct 14 16:53 loans.parquet
 47M -rw-r--r-- 1 root  47M Oct 14 16:51 loans.parquet.gzip


In [ ]:
%timeit df = pd.read_parquet('./loans.parquet')

2.76 s ± 227 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Predicate Push Down (filter while loading)**

In [ ]:
del df  ## free memory of the previous dataframe

df = pd.read_parquet('./loans.parquet', filters=[("date_created",">","2017-05-01 00:00:00"),("date_created","<","2017-06-01 00:00:00")])

In [ ]:
df.head()

,uuid,use_uuid,bicycle_uuid,anchor_uuid,date_created,type_access
0,3030296398be45ca9ec387d146c10ef2,024cbe02bf494e4f9125ede5a21db1a8,57ebb2215f2a45f0a1c0c5a97f6fde79,1042bbe5c80549c987e51f04d1cb9285,2017-05-11 18:03:53,UNKNOWN
1,30303bfde83e450e93580ca739f1cad2,None,1d06622f183d485297aa91a0da25ff40,64b4da4811624d77a3bf15225ed9a50f,2017-05-09 08:24:05,UNKNOWN
2,30304d9c5b1445a0a6cebd9a344b30bf,None,5813bd5f79b74cbfb267d164875ad08a,33677684d3c349c8b21d4303e838a1cd,2017-05-24 20:28:43,UNKNOWN
3,3030632449df4522acb7707f8e5191b3,178e8d38526347efb7c53bd01110fa70,388f697b5e904f4b9882fdbe432356e0,39fec48744294a4099102cd9f2800636,2017-05-06 16:25:29,UNKNOWN
4,3030667945d541b5964e72b9de3c10e1,None,e968ef5a4f61482a9c0b26b3dd174f14,ec917a85a7ab44659ab11ce702c830d8,2017-05-29 03:23:40,UNKNOWN


In [ ]:
df_taxi = pd.read_parquet('/content/yellow_tripdata_2023-01.parquet')

df_taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## PANDAS

**Exercise 1:** Using the parquet file, load only the column *date_created* for those rows with *use_uuid* different from None. With the loaded data, get the
total loans per [day of the week](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.dayofweek.html).

In [ ]:
## del df

**Exercise 2:** Using the parquet file, load only the column *bicycle_uuis* and get the top 100 most used bicycles along with their total loans.

In [ ]:
## del df

### POLARS

Cheat Sheet:
https://colab.research.google.com/github/FranzDiebold/polars-cheat-sheet/blob/main/polars-cheat-sheet.ipynb


In [ ]:
## Do the same steps with polars (except creating the parquet file)

### VAEX

Tutorial: https://vaex.io/docs/tutorials.html

In [ ]:
## Do the same steps with Vaex (except creating the parquet file)

## DUCKDB

https://duckdb.org/docs/sql/introduction.html

Python: https://duckdb.org/docs/api/python/overview.html

In [ ]:
## Do the same steps with DuckDB (except creating the parquet file)

### SUMMARY OF RESULTS

Report the execution times for the different frameworks:
- Loading CSV vs. Parquet
- Loading Parquet with filters
- Performing the required aggregations (Exercises 1 & 2)

In [ ]:
# Write a summary of the results obtained in the previous evaluations

## NEW YORK TRIP DATA

ORIGINAL DATA:
-   NYC Yellow Taxi (original): https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

KAGGLE (in CSV and only 2015):
-   Kaggle: https://www.kaggle.com/datasets/elemento/nyc-yellow-taxi-trip-data

DATA FOR THE EXERCISE (place these files into a folder and load the whole folder of parquets with "folder/*"):
-   Server: https://krono.act.uji.es/IDIA/yellow_tripdata_2015-01.parquet
-   Server: https://krono.act.uji.es/IDIA/yellow_tripdata_2023-01.parquet

**Exercise 3**: Compare the number of trips occurred in January 2015 and January 2023. Choose the best framework you consider for this task, and use predicates to get only what you need for the analysis.

In [ ]:
# your code here